# Benchmark creation with [Polaris](https://github.com/polaris-hub/polaris)

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import pathlib

import datamol as dm
import numpy as np

# polaris benchmark
from polaris.benchmark import SingleTaskBenchmarkSpecification

# polaris hub
from polaris.cli import PolarisHubClient
from polaris.utils.types import HubOwner

# utils
root = pathlib.Path("__file__").absolute().parents[2]
os.chdir(root)
sys.path.insert(0, str(root))

In [2]:
# Get the owner and organization
# Get the owner and organization
org = "AdaptyvBio"
data_name = "IL7Ra_binders"
dataset_version = "v1"
dirname = dm.fs.join(root, f"org-{org}",data_name)
gcp_root = f"gs://polaris-public/polaris-recipes/org-{org}/{data_name}"

owner = HubOwner(slug="adaptyv-bio", type="organization")
owner

BENCHMARK_DIR = f"{gcp_root}/benchmarks"
DATASET_JSON = f"gs://polaris-public/polaris-recipes/org-{org}/{data_name}/datasets/{data_name}-{dataset_version}/dataset.json"

FIGURE_DIR = f"{gcp_root}/figures"

## Load existing Dataset
> **Attention:** \
> See more dataset creataion details in [02_polaris_adme-fang_dataset.ipynb](https://github.com/polaris-hub/polaris-recipes/org-Biogen/fang2023_ADME/02_polaris_adme-fang_dataset.ipynb).

In [3]:
# Load the saved Dataset
import polaris as po

# Load the dataset from the Hub
dataset = po.load_dataset("adaptyv-bio/IL7Ra_binders-v1")

⠇ Fetching artifact... 

2024-09-12 16:53:42.888 | INFO     | polaris._artifact:_validate_version:66 - The version of Polaris that was used to create the artifact (0.7.9) is different from the currently installed version of Polaris (0.8.5.dev4+gd28c6e4).
2024-09-12 16:53:42.890 | INFO     | polaris.mixins._checksum:verify_checksum:65 - To verify the checksum, we need to recompute it. This can be slow for large datasets.


✅ SUCCESS: Fetched artifact.
 


/Users/lu.zhu/miniconda3/envs/po_dataset/lib/python3.12/site-packages/yaspin/core.py:228: UserWarning: color, on_color and attrs are not supported when running in jupyter
  self._color = self._set_color(value) if value else value


## Benchmark creation with `Polaris`
This is a retrospective benchmark, all the data points are in the test split for evaluation. 

In [8]:
split = (np.array([]), np.array(range(dataset.table.shape[0])))

In [9]:
split

(array([], dtype=float64),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
        51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
        68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
        85, 86, 87, 88, 89, 90, 91, 92, 93, 94]))

## Create a benchmark

In [10]:
benchmark_version = "v1"
target_cols = ["Binder_bool"]
input_col = ["sequence"]
benchmark_name = "IL7Ra_binders_binary_cls"


In [11]:
benchmark_name = f"{benchmark_name}-{benchmark_version}"
benchmark = SingleTaskBenchmarkSpecification(
    name=benchmark_name,
    dataset=dataset,
    target_cols=target_cols,
    target_types={"Binder_bool": "classification"},
    input_cols=input_col,
    split=split,
    main_metric="f1",
    metrics=["accuracy", "f1", "roc_auc", "pr_auc", "mcc", "cohen_kappa"],
    tags=["protein-design", "singletask"],
    owner=owner,
    description=f"Single task benchmark for protein binder design targeting the interleukin-7 receptor alpha chain.",
)

2024-09-12 16:54:13.057 | INFO     | polaris.benchmark._base:_validate_split:188 - This benchmark only specifies a test set. It will return an empty train set in `get_train_test_split()`


In [12]:
benchmark

name,IL7Ra_binders_binary_cls-v1
description,Single task benchmark for protein binder design targeting the interleukin-7 receptor alpha chain.
tags,protein-designsingletask
user_attributes,
owner,adaptyv-bio
polaris_version,0.8.5.dev4+gd28c6e4
target_cols,Binder_bool
input_cols,sequence
metrics,accuracyf1roc_aucpr_aucmcccohen_kappa
main_metric,f1
readme,


In [14]:

# upload to polaris hub
benchmark.upload_to_hub(access="private", owner=owner)

💥 ERROR: Failed to upload benchmark. 


/Users/lu.zhu/miniconda3/envs/po_dataset/lib/python3.12/site-packages/yaspin/core.py:228: UserWarning: color, on_color and attrs are not supported when running in jupyter
  self._color = self._set_color(value) if value else value


PolarisHubError: The request to the Polaris Hub failed. The Hub responded with:
{
  "issues": [
    {
      "code": "too_small",
      "exact": false,
      "inclusive": true,
      "message": "Array must contain at least 1 element(s)",
      "minimum": 1,
      "path": [
        "split",
        0
      ],
      "type": "array"
    }
  ]
}
